In [34]:
%load_ext autoreload
%autoreload 2
from mail_agent.app import init_app, MailAgentApp
from mail_agent.models import User
from mail_agent.gmail import Gmail
import json 
import pandas as pd
from tabulate import tabulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
c = pd.read_csv('/Users/eric/proj/ai-project/mail-agent/config/categories.csv')
print(tabulate(c, headers='keys', tablefmt='psql'))

+----+-----------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | title                       | name          | description                                                                                                                                                                                                |
|----+-----------------------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Cold Business Email         | cold          | Use for requests to consider a business promotion or service, such                                                                                                           

In [16]:
def extract_address(email):
    """Extracts the email address from a string."""
    if '<' in email and '>' in email:
        return email.split('<')[1].split('>')[0].strip()
    return email.strip()


In [19]:
app = init_app()

# Check if user needs re-authentication
rows = []
with app.app_context():
    
    first_user = User.query.first()

    for email in first_user.emails:
        if email.analysis:

            d = {
                'id': email.id,
                'subject': email.subject,
                'from': extract_address(email.from_address),
                'from_domain': extract_address(email.from_address).split('@')[-1],
                'to': extract_address(email.to_address),
                'to_domain': extract_address(email.to_address).split('@')[-1],
            }
            d.update(json.loads(email.analysis))
            rows.append(d)

In [25]:
df = pd.DataFrame(rows)
df[['id','categories']].explode('categories').categories.value_counts().to_csv('categories.csv')

In [27]:
df

,id,subject,from,from_domain,to,to_domain,summary,categories,is_reply_to_organization,is_cold_prospecting,is_promotion,is_business_operations,is_time_sensitive,confidence
0,1970dc85267c8e04,Re: Classes for Ashay and Devin Moorjani,jed.stumpf@jointheleague.org,jointheleague.org,seemakulkarni@yahoo.com,yahoo.com,"The email exchange involves a parent, Seema, d...","[Membership Cancellation, Education Program, C...",True,False,False,True,True,0.9
1,1970dc37e4b66ce4,Re: Classes for Ashay and Devin Moorjani,info@jointheleague.org,jointheleague.org,jed.stumpf@jointheleague.org,jointheleague.org,The email is a conversation between Seema and ...,"[Education, Membership Cancellation, Alumni En...",True,False,False,True,True,0.9
2,1970dae5ab094828,Re: Classes for Ashay and Devin Moorjani,jed.stumpf@jointheleague.org,jointheleague.org,seemakulkarni@yahoo.com,yahoo.com,Seema is informing Jed that her sons will be t...,"[Education, Membership Cancellation, Alumni En...",True,False,False,False,True,0.9
3,1970d6017d6eb6e3,Jed Stumpf sent you messages,notification@slack.com,slack.com,eric.busboom@jointheleague.org,jointheleague.org,The email is a notification from Slack informi...,"[Notification, Group Communication, Team Colla...",False,False,False,False,False,0.9
4,1970d4232a7df478,🖥️ Upgrade Your Gaming with Top Desktop PCs an...,Promo@promo.newegg.com,promo.newegg.com,eric.busboom@jointheleague.org,jointheleague.org,The email is a promotional message from Newegg...,"[Promotional, Technology, E-commerce, Discount...",False,False,True,False,True,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,196ea23bb770153d,Reminder: Call For Sessions Due May 23,community@jointheleague.org,jointheleague.org,community@jointheleague.org,jointheleague.org,The email is a reminder for submitting session...,"[Event Reminder, Collaboration Opportunity, Sp...",False,False,True,False,True,0.9
191,196d164416074d70,Changed: 45 Minute Meeting | Johanna Schnell -...,scheduler-noreply@asynccomm.zoom.us,asynccomm.zoom.us,eric.busboom@jointheleague.org,jointheleague.org,The email notifies Eric Busboom of a reschedul...,"[Meeting Rescheduling, Zoom Meeting Invitation...",False,False,False,False,True,0.9
192,196d072f2f862a12,Booked: 45 Minute Meeting | Lynne Broyles - Fr...,scheduler-noreply@asynccomm.zoom.us,asynccomm.zoom.us,eric.busboom@jointheleague.org,jointheleague.org,The email is a meeting invitation for a 45-min...,"[Meeting Invitation, Zoom Meeting, Business Co...",False,False,False,False,True,0.9
193,196cfb262a40b75d,Booked: 45 Minute Meeting | Johanna Schnell - ...,scheduler-noreply@asynccomm.zoom.us,asynccomm.zoom.us,eric.busboom@jointheleague.org,jointheleague.org,Meeting invitation from Johanna Schnell for a ...,"[Meeting Invitation, Zoom Meeting, Calendar Ev...",False,False,False,False,True,0.9
